We load the data with the features.
The booked column is the label.
We hot encode the categorical feature cabin_class and drop one.
To tune the classifier, we divide the data into training, testing and validation sets.

In [ ]:
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics


df = pd.read_csv('features_classification.csv')
df = df.drop(['booking_id'], axis=1) 
y = df['booked'].values
X = df.drop('booked',1)
dummies = pd.get_dummies(X[['cabin_class']])
X = pd.concat([X, dummies], axis=1)
X = X.drop(['cabin_class','cabin_class_mixed'], 1)

X_, X_test, y_, y_test = train_test_split(X,y, test_size=0.40, random_state=42)#[:20000] 
X_train, X_valid, y_train, y_valid = train_test_split(X_ ,y_ , test_size=0.40, random_state=42)

I chose to use a svm classifier for the task since the processing is much more faster.
I looked at the precision, recall and f1 metrics on the testing dataset to see how to tune the classifier.
I also chose to add a weight to the classes since the booked class is under-represented, it then forces the classifier to concentrate on the small class.
I also tried to find the best combination of parameters with a search grid even if the computation time is important.

In [ ]:
class_weight = {1: 30, 0: 1}

clf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=10, tol=None,class_weight=class_weight)
clf.fit(X_train,y_train)
predicted_SVM = clf.predict(X_test)
print("SVM part, metrics on test set:")
print(metrics.classification_report(y_test, predicted_SVM))

#from sklearn.model_selection import GridSearchCV
#parameters = {'max_iter':(5,10,15),'alpha': (1e-2, 1e-3),'class_weight':({1: 10, 0: 1},{1: 20, 0: 1},{1: 30, 0: 1})}
#gs_clf = GridSearchCV(clf, parameters, n_jobs=-1)
#gs_clf = gs_clf.fit(X_train, y_train)
#print(gs_clf.best_score_)                                 
#
#for param_name in sorted(parameters.keys()):
#    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

To be sure that I don't tune in too much the model, I predicted some data that the model has never see.

In [ ]:

predicted_valid_SVM = clf.predict(X_valid)
print("SVM part, metrics on validation set:")
print(metrics.classification_report(y_valid, predicted_valid_SVM))

For this problem, I basically considered the same set of features as for the regression problem, I can imagine that not all of these are that relevant for this task. We are here trying to understand the decisions of a user, I would thus try to look more into that direction maybe by working on the session_ids or the user_agent. Something that I haven't considered at all. 